In [1]:
import pandas as pd
home_dir="/bml/shreya/TF_binding_site/dataset_test/DEEPSEA_dataextraction/"
print(home_dir)

/bml/shreya/TF_binding_site/dataset_test/DEEPSEA_dataextraction/


  Using cached pandas-2.2.3-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (89 kB)
  Using cached pytz-2025.2-py2.py3-none-any.whl.metadata (22 kB)
Using cached pandas-2.2.3-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (13.1 MB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 40.1 MB/s eta 0:00:0000:010:01
Using cached pytz-2025.2-py2.py3-none-any.whl (509 kB)
Note: you may need to restart the kernel to use updated packages.


In [2]:
fpath = home_dir+"data/processed/merged.sorted.gz"
print(fpath)
col_names = ["genomeChrom", "genomeChromStart", "genomeChromEnd", "filepath", "chrom", "chromStart", "chromEnd", "name", "score", "strand", "signalValue", "pValue", "qValue", "peak", "intersectedBps"] 
peaks_df = pd.read_csv(fpath, sep="\t", names=col_names, compression='gzip') # ~50 seconds to load
print(peaks_df.shape)
peaks_df

/bml/shreya/TF_binding_site/dataset_test/DEEPSEA_dataextraction/data/processed/merged.sorted.gz
(19730705, 15)


,genomeChrom,genomeChromStart,genomeChromEnd,filepath,chrom,chromStart,chromEnd,name,score,strand,signalValue,pValue,qValue,peak,intersectedBps
0,chr1,10000,10200,wgEncodeAwgTfbsHaibK562Zbtb33Pcr1xUniPk,chr1,10073,10329,.,138,.,21.646309,-1,2.681652,128,NaN
1,chr1,10200,10400,wgEncodeAwgTfbsHaibK562Zbtb33Pcr1xUniPk,chr1,10073,10329,.,138,.,21.646309,-1,2.681652,128,NaN
2,chr1,10200,10400,wgEncodeAwgTfbsSydhH1hescCebpbIggrabUniPk,chr1,10149,10413,.,201,.,31.517985,-1,2.543785,132,NaN
3,chr1,16200,16400,wgEncodeAwgTfbsUtaHelas3CtcfUniPk,chr1,16165,16325,.,139,.,21.832632,-1,4.550656,80,NaN
4,chr1,16200,16400,wgEncodeAwgTfbsUtaK562CtcfUniPk,chr1,16155,16339,.,171,.,26.766516,-1,5.104183,92,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19730700,chrX,155260200,155260400,wgEncodeAwgTfbsHaibH1hescPol24h8V0416102UniPk,chrX,155260046,155260456,.,243,.,38.095921,-1,5.029931,205,NaN
19730701,chrX,155260200,155260400,wgEncodeAwgTfbsHaibSknshPol24h8V0416101UniPk,chrX,155260029,155260473,.,273,.,42.759047,-1,4.871719,222,NaN
19730702,chrX,155260200,155260400,wgEncodeAwgTfbsHaibGm12878Tcf3Pcr1xUniPk,chrX,155260125,155260429,.,129,.,20.213742,-1,4.708871,152,NaN
19730703,chrX,155260400,155260600,wgEncodeAwgTfbsHaibGm12891Pol24h8Pcr1xUniPk,chrX,155260001,155260511,.,268,.,42.049623,-1,4.989650,255,NaN


In [3]:
# dropping chromosomal coordinate duplicates with labels (filepath), because same chromosomal coordinate may have multiple overlapping
# ie. chrom coord: chr1 1 100, 1st peak 20-80, 2nd peak 30-90. Both of this peaks result in same label for the same chomosomal coordinate.
x = peaks_df[["genomeChrom", "genomeChromStart", "genomeChromEnd", "filepath"]].drop_duplicates(ignore_index=True, inplace=False)
print(x.shape)

# aggregating the labels (comma and subsequent space separated)
y = x.groupby(["genomeChrom", "genomeChromStart", "genomeChromEnd"]).filepath.agg([('count', 'count'), ('filepath', ', '.join)]).reset_index()
# y[y["count"]==2].loc[26]["filepath"]
print(y.shape)

(18896347, 4)
(1903712, 5)


In [4]:
y.to_csv(home_dir+"data/processed/peaks_with_labels.tsv.gz", compression="gzip", sep="\t", index=False, header=False)

In [12]:
y.head()

,genomeChrom,genomeChromStart,genomeChromEnd,count,filepath
0,chr1,10000,10200,1,wgEncodeAwgTfbsHaibK562Zbtb33Pcr1xUniPk
1,chr1,10200,10400,2,"wgEncodeAwgTfbsSydhH1hescCebpbIggrabUniPk, wgE..."
2,chr1,16200,16400,7,"wgEncodeAwgTfbsUtaMcf7CtcfSerumstimUniPk, wgEn..."
3,chr1,29200,29400,2,"wgEncodeAwgTfbsHaibHepg2GabpPcr2xUniPk, wgEnco..."
4,chr1,29400,29600,2,"wgEncodeAwgTfbsHaibHepg2GabpPcr2xUniPk, wgEnco..."


In [5]:
y["count"].value_counts()

count
1      686782
2      295035
3      174646
4      117902
5       82779
        ...  
426         1
366         1
399         1
393         1
320         1
Name: count, Length: 382, dtype: int64

In [6]:
y[["genomeChrom", "genomeChromStart", "genomeChromEnd", "filepath"]].value_counts()

genomeChrom  genomeChromStart  genomeChromEnd  filepath                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                 

In [7]:
peaks_df[(peaks_df["genomeChrom"]=="chr12") & (peaks_df["genomeChromStart"]==54145200)] #, "genomeChromEnd"]]

,genomeChrom,genomeChromStart,genomeChromEnd,filepath,chrom,chromStart,chromEnd,name,score,strand,signalValue,pValue,qValue,peak,intersectedBps
4311519,chr12,54145200,54145400,wgEncodeAwgTfbsBroadNhaEzh239875UniPk,chr12,54144430,54146352,.,329,.,51.530607,-1,4.167406,1588,NaN
4311520,chr12,54145200,54145400,wgEncodeAwgTfbsHaibGm12892Pax5c20V0416101UniPk,chr12,54145184,54145464,.,255,.,39.979920,-1,4.423885,140,NaN
4311521,chr12,54145200,54145400,wgEncodeAwgTfbsBroadH1hescEzh239875UniPk,chr12,54145128,54146257,.,620,.,97.096848,-1,3.917558,617,NaN
4311522,chr12,54145200,54145400,wgEncodeAwgTfbsSydhGm18951Pol2IggmusUniPk,chr12,54145191,54145681,.,178,.,28.014719,-1,4.690683,245,NaN
4311523,chr12,54145200,54145400,wgEncodeAwgTfbsSydhGm12878Chd2ab68301IggmusUniPk,chr12,54145186,54145566,.,209,.,32.718212,-1,2.467111,190,NaN
4311524,chr12,54145200,54145400,wgEncodeAwgTfbsBroadNhaEzh239875UniPk,chr12,54144430,54146352,.,633,.,99.090649,-1,4.167406,1186,NaN
4311525,chr12,54145200,54145400,wgEncodeAwgTfbsSydhNt2d1Suz12UcdUniPk,chr12,54145193,54146003,.,179,.,28.054665,-1,3.754119,405,NaN
4311526,chr12,54145200,54145400,wgEncodeAwgTfbsBroadNhlfEzh239875UniPk,chr12,54145006,54145626,.,315,.,49.374896,-1,2.764528,310,NaN
4311527,chr12,54145200,54145400,wgEncodeAwgTfbsSydhGm12878Mxi1IggmusUniPk,chr12,54145227,54145663,.,167,.,26.175163,-1,0.450162,218,NaN
4311528,chr12,54145200,54145400,wgEncodeAwgTfbsBroadDnd41Ezh239875UniPk,chr12,54145131,54146005,.,263,.,41.255267,-1,2.995489,702,NaN
